# Random Ensemble Orchestration - With get STATE


Global Variables


In [25]:
from model_rl import ServiceConfig, ProcessingService, ServiceType, InferenceService, EnsembleService
import logging
import os
import time
import pandas as pd
import random
import copy
from collections import Counter
from rohe.common import rohe_utils as utils
from IPython.display import clear_output
from config_manager import ConfigManager
import yaml

# Configure logging
#logging.basicConfig(format="%(asctime)s:%(levelname)s -- %(message)s", level=logging.INFO)
#logging.getLogger().setLevel(logging.CRITICAL)

# Paths and configurations
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
MODEL_DATA_PATH = os.path.join(parent_dir, "profile/processed")
PROFILE_PATH = os.path.join(parent_dir, "profile/model_profile/model_profile.yaml")
CONFIG_PATH = os.path.join(current_dir, "sim_config.yaml")
FILE_DATA_PATH = os.path.join(current_dir, "file_label.csv")
THROUGHPUT_REQUIREMENT = 15

def setup_config():
    # RELOAD INITIAL YAML
    initial_yaml_file_path = "initial_sim_config.yaml"  # Ensure this path matches your desired location
    # Load contents from the initial YAML file
    with open(initial_yaml_file_path, "r") as initial_file:
        initial_yaml_content = yaml.safe_load(initial_file)

    # Write the loaded contents into the target YAML file
    with open(CONFIG_PATH, "w") as target_file:
        yaml.dump(initial_yaml_content, target_file, default_flow_style=False)

    print(f"Contents of {initial_yaml_file_path} have been dumped into {CONFIG_PATH}.")

    # Load necessary data
    profile_data = utils.load_config(PROFILE_PATH)
    data_file_label = pd.read_csv(FILE_DATA_PATH).groupby("label")
    labels = list(data_file_label.groups.keys())
    model_profile_data = {}

setup_config()

# Input data example
init_data = {
    "input": {
        "file_name": "n01560419_3101",
        "image_height": 224,
        "image_width": 224,
    }
}

weights = {
    "accuracy": 1.0,
    "confidence": 0.0,
    "latency": 0.0,
    "energy": 0.0,
    "explainability": 0.0
}


# Global variables
sim_config = None
distribution_keys = None
distribution_weights = None
processing_service = None
ensemble_service = None
total_energy_consumption = 0






Contents of initial_sim_config.yaml have been dumped into c:\Documents\Aalto Semester 1\Research\ROHE_orchestrator\ROHE_orchestrator\source\rl_simulation\sim_config.yaml.


## Define necessary functions

In [26]:
# Function to configure the pipeline
def config_pipeline():
    global sim_config, distribution_keys, distribution_weights, processing_service, ensemble_service

    # Load simulation config
    sim_config = utils.load_config(CONFIG_PATH)

    # Input distribution
    distribution = sim_config["distribution"]
    distribution_keys = list(distribution.keys())
    distribution_weights = list(distribution.values())

    # Create processing service
    if "processing" in sim_config:
        processing_config = sim_config["processing"]
        processing_config["service_type"] = ServiceType.PROCESSING.value
        processing_service_config = ServiceConfig.model_validate(processing_config)
        processing_service = ProcessingService(processing_service_config)

    # Create ensemble service
    if "ensemble" in sim_config:
        ensemble_config = sim_config["ensemble"]
        ensemble_config["service_type"] = ServiceType.ENSEMBLE.value
        ensemble_service_config = ServiceConfig.model_validate(ensemble_config)
        ensemble_service = EnsembleService(ensemble_service_config)
        #ensemble_service.load_models_from_yaml(sim_config["throughput_requirement"], PROFILE_PATH)




    # Add inference services to ensemble
    if "inference" in sim_config:
        inference_configs = sim_config["inference"]
        for model_name, model_config in inference_configs.items():
            model_config["throughput"] = profile_data[model_name]["throughput"]
            model_config["energy"] = profile_data[model_name]["energy"]
            model_config["response_time"] = profile_data[model_name]["response_time"]
            model_config["service_type"] = ServiceType.INFERENCE.value
            model_config["data_path"] = str(MODEL_DATA_PATH)
            model_config["throughput_requirement"] = int(sim_config["throughput_requirement"])
            inference_service_config = ServiceConfig.model_validate(model_config)
            inference_service = InferenceService(inference_service_config)
            ensemble_service.add_model(inference_service)



# Function to simulate applying RL agent actions
def apply_action(action):
    manager = ConfigManager(PROFILE_PATH, CONFIG_PATH)

    if action == "keep_ensemble":
        logging.info("Action: Keeping the ensemble")
    
    elif action == "add_model":
        manager.add_best_model(weights)
        logging.info("Action: Adding a model")
        # Logic for adding a new model (Placeholder)
    elif action == "replace_model":
        logging.info("Action: Replacing a model")
        manager.remove_worst_model(weights)
        manager.add_best_model(weights)
        # Logic for replacing a model (Placeholder)
    elif action == "remove_model":
        manager.remove_worst_model(weights)

    elif action == "add_random_model":
        manager.add_random_model()
    elif action == "remove_random_model":
        manager.remove_random_model()
    elif action == "replace_random_model":
        manager.remove_random_model()
        manager.add_random_model()


    else:
        logging.warning("Unknown action")



In [33]:
### DEFINE GET STATE RL

# IMAGE DATA +
# ENSEMBLE MDOELS DATA ( CAPPED AT 10)

# FLATTEN

import numpy as np

def get_state(data):
    # Ensemble-level metrics
    energy_report = ensemble_service.energy_estimate()
    ensemble_state = {
        "total_energy_consumption": total_energy_consumption + energy_report["ensemble"],
        "ensemble_size": len(ensemble_service.ensemble),
    }

    # Model-level metrics
    model_states = []
    for model_name, model_data in model_profile_data.items():
        # Directly compute metrics from data_frame
        if "data_frame" in model_data:
            recent_df = model_data["data_frame"].tail(10000)  # Use recent 10,000 entries
            accuracy = recent_df["accuracy"].mean()
            confidence = recent_df["confidence"].mean()
            avg_response_time = recent_df["response_time"].mean()
            max_response_time = recent_df["response_time"].max()
            contribution = recent_df["contribution"].mean()

            model_states.append([
                accuracy,
                confidence,
                avg_response_time,
                max_response_time,
                contribution
            ])

    # Ensure fixed-length state for 10 models
    max_models = 10
    num_features = 5  # Accuracy, Confidence, Avg Response, Max Response, Contribution
    while len(model_states) < max_models:
        model_states.append([0.0] * num_features)  # Padding with zeros

    # Truncate if more than 10 models (just in case)
    model_states = model_states[:max_models]

    # Input state (flattened input metrics)
    input_state = [
        len(data["input"]["file_name"]),  # Example input length
        data["input"]["image_height"],
        data["input"]["image_width"],
    ]

    # Ensemble state metrics
    ensemble_state_vector = [
        ensemble_state["total_energy_consumption"],
        ensemble_state["ensemble_size"],
    ]
    """
    # Combine and flatten state
    state = np.concatenate([
        np.array(ensemble_state_vector, dtype=np.float32),
        np.array(model_states, dtype=np.float32).flatten(),
        np.array(input_state, dtype=np.float32)
    ]) 
    """
    structured_state = {
        "ensemble_state": ensemble_state,
        "model_states": model_states,
        "input_state": input_state
    }


    return structured_state

def flatten_structured_state(structured_state):
    """
    Flattens the structured state dictionary into a single NumPy array.

    Args:
        structured_state (dict): State dictionary with keys:
            - "ensemble_state": dict with ensemble metrics.
            - "model_states": list of model-level metrics.
            - "input_state": list of input-level metrics.

    Returns:
        np.array: Flattened state array.
    """
    # Ensemble state as a vector
    ensemble_state_vector = [
        structured_state["ensemble_state"]["total_energy_consumption"],
        structured_state["ensemble_state"]["ensemble_size"],
        *distribution_weights

    ]

    # Model states as a flattened vector
    model_states_vector = []
    for model in structured_state["model_states"]:
        model_states_vector.extend(model)  # Assuming model states are lists of 5 metrics

    # Input state as a vector
    input_state_vector = structured_state["input_state"]

    # Combine all parts into a single flattened array
    flattened_state = np.concatenate([
        np.array(ensemble_state_vector, dtype=np.float32),
        np.array(model_states_vector, dtype=np.float32),
        np.array(input_state_vector, dtype=np.float32)
    ])

    return flattened_state




In [34]:
def random_data_metrics():
    # returns data and inference metrics on a random image
    config_pipeline()

    model_count = 0
    # Prepare input data: 224-224
    data = copy.deepcopy(init_data)
    
    # SELECT THE INPUT IMAGE
    selected_key = random.choices(distribution_keys, weights=distribution_weights, k=1)[0]
    if selected_key in labels:
        df_file = data_file_label.get_group(selected_key)
        file_name = random.choice(df_file["file_name"].values)
    data["input"]["file_name"] = file_name
    data["label"] = selected_key
    
    
    # Execute services
    data = processing_service.execute(data)
    data = ensemble_service.execute(data)
    
    
    # Update model profiles
    for model_name, inferences in data["ml_inference"].items():
        model_count +=1 
        response_time = data["response_time"]["inference"].get(model_name, 0)

        i_label = data["label"]
        i_accuracy = 1 if i_label in inferences else 0
        i_confidence = inferences.get(i_label, 0)
        model_contribution = data["contribution"].get(model_name, 0)


        # Save to profile data
        data_dict = {
            "label": [i_label],
            "accuracy": [i_accuracy],
            "confidence": [i_confidence],
            "response_time": [response_time],
            "contribution": [model_contribution],
        }
        result_df = pd.DataFrame(data_dict)
        if model_name not in model_profile_data:
            model_profile_data[model_name] = {"data_frame": result_df}
        else:
            model_profile_data[model_name]["data_frame"] = pd.concat(
                [model_profile_data[model_name]["data_frame"], result_df], ignore_index=True
            )

    return data
    

## RL test

Defining the Simulation enviroment

In [35]:
import gym
from gym import spaces
import numpy as np

#substitute everything with get state

# ADD DISTRIBUTION WEIGHTS TO THE SIMULATION ENV

class SimulationEnv(gym.Env):

    def __init__(self):
        super(SimulationEnv, self).__init__()

        # Define constants
        self.state_size = 55 + 20  # Change this to be dynamic based on the enviroment
        # Observation space: Fixed size of 55
        self.observation_space = spaces.Box(
            low=0.0, high=10.0, shape=(self.state_size,), dtype=np.float32
        )

        # Action space (example: add/remove/replace a model)
        self.action_space = spaces.Discrete(6)

    def reset(self):
        # Reset the environment to an initial state

        # Define Init Data:
        data = random_data_metrics()

        self.current_state = get_state(data)  # Get initial state
        print(self.current_state)
        return flatten_structured_state(self.current_state)

    def step(self, action):
        
                
        if action == 0:
            apply_action("keep_ensemble")
        elif action == 1:
            apply_action("add_model")
        elif action == 2:
            apply_action("replace_model")


        data = random_data_metrics()
        state = get_state(data)

        # Simulate a step in the environment
        self.current_state = state
        reward = self._calculate_reward(self.current_state)
        print("STATE:" , state)
        done = False  # Define termination condition if applicable
       
        return flatten_structured_state(self.current_state), reward, done, {}

    def _state_to_observation(self, state):
        # Convert the state dictionary to a flat vector for the RL agent
        return np.array(list(state["ensemble"].values()) + [model["accuracy"] for model in state["models"]])

    def _calculate_reward(self, state):
        model_accuracies = [model[0] for model in state.get("model_states", []) if model[0]!= 0]  # Index 0 -> Accuracy
        
        # Reward is the mean accuracy; default to 0 if no accuracies exist
        if model_accuracies:
            accuracy_reward = np.mean(model_accuracies)
        else:
            accuracy_reward = 0.0

        # Debugging output
        print("Model Accuracies:", model_accuracies)
        print("Accuracy Reward:", accuracy_reward)

        return accuracy_reward

Running the simulation

In [38]:
from stable_baselines3 import PPO

setup_config()
# Create the environment
env = SimulationEnv()

# Train the agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1)

# Save the trained model
model.save("simulation_policy")


Contents of initial_sim_config.yaml have been dumped into c:\Documents\Aalto Semester 1\Research\ROHE_orchestrator\ROHE_orchestrator\source\rl_simulation\sim_config.yaml.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Filippo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


{'ensemble_state': {'total_energy_consumption': 0.00329458735984739, 'ensemble_size': 3}, 'model_states': [[0.920388349514563, 0.8149167671365646, 2.073909771218895, 2.5248890024990285, 0.7917360034093116], [0.8219512195121951, 0.6646395952236361, 0.09457568744741897, 0.1964937604702088, 0.6553794139042133], [0.7097701149425287, 0.6497511538280838, 0.38968308142394287, 0.5163422452602895, 0.7171673640266232], [0.8825242718446602, 0.7485794605604142, 6.916381610661182, 7.288435090373072, 0.9041109834191868], [0.9576771653543307, 0.8398253469485936, 1.1472186519699605, 1.3322012301546087, 0.7966453300451669], [0.9469469469469469, 0.8303964472151136, 3.819046791436031, 4.001362883372639, 0.794377355335711], [0.9459183673469388, 0.7106480975540317, 7.1196906674063785, 7.279399427503429, 0.676256727929018], [0.9085239085239085, 0.8117234781229571, 0.5848439618190212, 0.835027244945422, 0.7967551179276683], [0.8827292110874201, 0.7385769925519093, 0.9938437923720507, 1.1020057401462464, 0.72

2024-12-19 18:07:35,304:INFO -- Action: Adding a model


Model Accuracies: [0.9204655674102813, 0.8221680876979294, 0.7101865136298422, 0.8826382153249273, 0.9576771653543307, 0.9469469469469469, 0.9459183673469388, 0.9085239085239085, 0.8827292110874201, 0.8926615553121577]
Accuracy Reward: 0.8869915538634683
STATE: {'ensemble_state': {'total_energy_consumption': 0.00329458735984739, 'ensemble_size': 3}, 'model_states': [[0.9204655674102813, 0.8150326349455448, 2.073876825658556, 2.5248890024990285, 0.7918743549852436], [0.8221680876979294, 0.6649922529614364, 0.0944867745209515, 0.1964937604702088, 0.6557433507910019], [0.7101865136298422, 0.6502504853299221, 0.3892874543650033, 0.5163422452602895, 0.7175699721278898], [0.8826382153249273, 0.7487865717486072, 6.911659269376788, 7.288435090373072, 0.9042018410595266], [0.9576771653543307, 0.8398253469485936, 1.1472186519699605, 1.3322012301546087, 0.7966453300451669], [0.9469469469469469, 0.8303964472151136, 3.819046791436031, 4.001362883372639, 0.794377355335711], [0.9459183673469388, 0.71

...

2024-12-19 18:07:48,398:INFO -- Action: Replacing a model


Model Accuracies: [0.9218455743879472, 0.8218181818181818, 0.71, 0.8822975517890772, 0.958930276981853, 0.9484936831875608, 0.9464285714285714, 0.9079878665318504, 0.8838174273858921, 0.893048128342246]
Accuracy Reward: 0.887466726185318
STATE: {'ensemble_state': {'total_energy_consumption': 0.018556997220561713, 'ensemble_size': 11}, 'model_states': [[0.9218455743879472, 0.8149709926879564, 2.0740752243769, 2.5248890024990285, 0.7918381072201971], [0.8218181818181818, 0.6647843555248145, 0.09426920980598333, 0.1964937604702088, 0.6565080694718795], [0.71, 0.6502920323610306, 0.3881096431155094, 0.5163422452602895, 0.7187434548991067], [0.8822975517890772, 0.7484972564806436, 6.907197398915765, 7.288435090373072, 0.9041276024391423], [0.958930276981853, 0.8400405684651482, 1.1472829434461518, 1.3322012301546087, 0.796006632312321], [0.9484936831875608, 0.8305664085090682, 3.81897779768301, 4.001362883372639, 0.7935799326678871], [0.9464285714285714, 0.709828064939569, 7.119861796202502

2024-12-19 18:07:49,074:INFO -- Action: Replacing a model


Model Accuracies: [0.9219190968955786, 0.8218181818181818, 0.71, 0.8824082784571966, 0.958969465648855, 0.9485436893203884, 0.9464816650148662, 0.908080808080808, 0.883937823834197, 0.8931623931623932]
Accuracy Reward: 0.8875321402232466
STATE: {'ensemble_state': {'total_energy_consumption': 0.018556997220561713, 'ensemble_size': 11}, 'model_states': [[0.9219190968955786, 0.8150681553028265, 2.074107174636045, 2.5248890024990285, 0.7919570317216785], [0.8218181818181818, 0.6647843555248145, 0.09426920980598333, 0.1964937604702088, 0.6565080694718795], [0.71, 0.6502920323610306, 0.3881096431155094, 0.5163422452602895, 0.7187434548991067], [0.8824082784571966, 0.7485113805246999, 6.907433319952965, 7.288435090373072, 0.9041408924201919], [0.958969465648855, 0.8400055601169135, 1.1472455766297291, 1.3322012301546087, 0.7960136410787361], [0.9485436893203884, 0.830633903070561, 3.81895590265309, 4.001362883372639, 0.7936833364292256], [0.9464816650148662, 0.709754074972614, 7.1198888611168

KeyboardInterrupt: 